In [1]:
import os
import glob
import numpy as np
import pandas as pd
import joblib
import itertools
import warnings
from tqdm import tqdm
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_score, train_test_split, RepeatedKFold

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline

# Modelos del Zoo
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, HuberRegressor, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict

# Nuevos modelos de boosting
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from src.utils import LoadData

warnings.filterwarnings("ignore")
DIRECTORIO_SALIDA = "../modelos_ajustados"
os.makedirs(DIRECTORIO_SALIDA, exist_ok=True)

In [2]:
def get_model_pool(random_state=42):
    """
    Pool de modelos con regularización ULTRA-CONSERVADORA
    """
    return {
        'ExtraTrees': ExtraTreesRegressor(
            n_estimators=30,
            max_depth=6,
            min_samples_leaf=30,
            min_samples_split=60,
            max_features=0.5,
            n_jobs=-1,
            random_state=random_state
        ),

        'RandomForest': RandomForestRegressor(
            n_estimators=30,
            max_depth=6,
            min_samples_leaf=30,
            min_samples_split=60,
            max_features=0.5,
            n_jobs=-1,
            random_state=random_state
        ),

        'DT-Simple': DecisionTreeRegressor(
            max_depth=3,
            min_samples_leaf=30,
            min_samples_split=60,
            random_state=random_state
        ),

        'Ridge': make_pipeline(
            StandardScaler(),
            Ridge(alpha=1.0)
        ),

        'KNN': make_pipeline(
            StandardScaler(),
            KNeighborsRegressor(
                n_neighbors=9,
                weights='distance',
                n_jobs=-1
            )
        ),

        # XGBoost con regularización MUY CONSERVADORA
        'XGBoost': XGBRegressor(
            n_estimators=60,
            max_depth=3,
            min_child_weight=15,
            learning_rate=0.05,
            subsample=0.7,
            colsample_bytree=0.7,
            reg_alpha=0.5,
            reg_lambda=2.0,
            n_jobs=-1,
            random_state=random_state,
            verbosity=0
        ),

        # LightGBM con regularización MUY CONSERVADORA
        'LightGBM': LGBMRegressor(
            n_estimators=60,
            max_depth=3,
            num_leaves=7,
            min_child_samples=35,
            learning_rate=0.05,
            feature_fraction=0.7,
            bagging_fraction=0.7,
            bagging_freq=5,
            reg_alpha=0.5,
            reg_lambda=2.0,
            min_gain_to_split=0.05,
            n_jobs=-1,
            random_state=random_state,
            verbose=-1,
            force_col_wise=True
        ),

        # ElasticNet
        'ElasticNet': make_pipeline(
            StandardScaler(),
            ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=2000)
        )
    }

"\n        # XGBoost con regularización MUY CONSERVADORA\n        'XGBoost': XGBRegressor(\n            n_estimators=60,\n            max_depth=3,\n            min_child_weight=15,\n            learning_rate=0.05,\n            subsample=0.7,\n            colsample_bytree=0.7,\n            reg_alpha=0.5,\n            reg_lambda=2.0,\n            n_jobs=-1,\n            random_state=random_state,\n            verbosity=0\n        ),\n\n        # LightGBM con regularización MUY CONSERVADORA\n        'LightGBM': LGBMRegressor(\n            n_estimators=60,\n            max_depth=3,\n            num_leaves=7,\n            min_child_samples=35,\n            learning_rate=0.05,\n            feature_fraction=0.7,\n            bagging_fraction=0.7,\n            bagging_freq=5,\n            reg_alpha=0.5,\n            reg_lambda=2.0,\n            min_gain_to_split=0.05,\n            n_jobs=-1,\n            random_state=random_state,\n            verbose=-1,\n            force_col_wise=True\n    

In [3]:
def get_param_grid(nombre_modelo, n_samples, n_features):
    """
    Grids de hiperparámetros ADAPTATIVOS según tamaño y dimensión del dataset
    Regularización más agresiva en datasets pequeños
    """
    # Clasificación del dataset
    is_small = n_samples < 500
    is_medium = 500 <= n_samples < 2000
    is_high_dim = n_features > 50

    # KNN: mínimo 5 vecinos, máximo razonable
    max_k = min(31, max(9, int(np.sqrt(n_samples))))

    grids = {

        'ExtraTrees': {
            'n_estimators': [20, 30, 50],
            'max_depth': (
                [3, 4, 5] if is_small else      # MUY bajo para pequeños
                [4, 5, 6] if is_medium else
                [5, 6, 8]
            ),
            'min_samples_leaf': (
                [30, 40, 50] if is_small else
                [25, 30, 35] if is_medium else
                [20, 25, 30]
            ),
            'min_samples_split': (
                [60, 80, 100] if is_small else
                [50, 60, 70] if is_medium else
                [40, 50, 60]
            ),
            'max_features': [0.3, 0.5, 0.7]
        },

        'RandomForest': {
            'n_estimators': [20, 30, 50],
            'max_depth': (
                [3, 4, 5] if is_small else
                [4, 5, 6] if is_medium else
                [5, 6, 8]
            ),
            'min_samples_leaf': (
                [30, 40, 50] if is_small else
                [25, 30, 35] if is_medium else
                [20, 25, 30]
            ),
            'min_samples_split': (
                [60, 80, 100] if is_small else
                [50, 60, 70] if is_medium else
                [40, 50, 60]
            ),
            'max_features': [0.3, 0.5, 0.7]
        },

        'DT-Simple': {
            'max_depth': (
                [2, 3, 4] if is_small else
                [3, 4, 5] if is_medium else
                [4, 5, 6]
            ),
            'min_samples_leaf': (
                [30, 40, 50] if is_small else
                [25, 30, 35] if is_medium else
                [20, 25, 30]
            ),
            'min_samples_split': (
                [60, 80, 100] if is_small else
                [50, 60, 70] if is_medium else
                [40, 50, 60]
            )
        },

        # Pipeline: StandardScaler + Ridge
        'Ridge': {
            'ridge__alpha': np.logspace(-3, 2, 25).tolist() # Antes empezaba en -2
        },

        # Pipeline: StandardScaler + KNN
        'KNN': {
            'kneighborsregressor__n_neighbors': list(range(5, max_k + 1, 2)),
            'kneighborsregressor__weights': ['uniform', 'distance'],
            'kneighborsregressor__p': [1, 2]
        },

        # XGBoost - Con penalización extra para datasets problemáticos
        'XGBoost': {
            'n_estimators': [40, 60, 80],
            'max_depth': (
                [2] if is_small else                    # SOLO depth=2 para boston
                [3, 4] if is_medium else
                [4, 5]
            ),
            'min_child_weight': (
                [15, 20, 25] if is_small else           # MÁS restrictivo para boston
                [8, 12, 15] if (is_medium and is_high_dim) else  # us_crime
                [5, 10] if is_medium else
                [3, 5]
            ),
            'learning_rate': (
                [0.01, 0.03] if is_small else           # Solo LR bajo para boston
                [0.02, 0.03, 0.05] if is_high_dim else  # us_crime
                [0.03, 0.05, 0.08]
            ),
            'subsample': [0.6, 0.7, 0.8],
            'colsample_bytree': (
                [0.4, 0.5] if is_high_dim else          # us_crime: solo 40-50% features
                [0.5, 0.6, 0.7]
            ),
            'reg_alpha': (
                [1.0, 2.0, 4.0] if (is_small or is_high_dim) else  # Regularización extrema
                [0.1, 0.5, 1.0]
            ),
            'reg_lambda': (
                [2.0, 4.0, 6.0] if (is_small or is_high_dim) else
                [1.0, 2.0, 4.0]
            )
        },

        # LightGBM - Similar ajuste
        'LightGBM': {
            'n_estimators': [40, 60, 80],
            'max_depth': (
                [2] if is_small else
                [3, 4] if is_medium else
                [4, 5]
            ),
            'num_leaves': (
                [3, 5] if is_small else                 # Menos hojas para boston
                [5, 7] if is_high_dim else              # us_crime
                [7, 10, 15] if is_medium else
                [15, 20, 25]
            ),
            'min_child_samples': (
                [30, 40, 50] if is_small else           # Más muestras para boston
                [25, 35] if (is_medium and is_high_dim) else  # us_crime
                [20, 25] if is_medium else
                [15, 20]
            ),
            'learning_rate': (
                [0.01, 0.03] if is_small else
                [0.02, 0.03, 0.05] if is_high_dim else
                [0.03, 0.05, 0.08]
            ),
            'feature_fraction': (
                [0.3, 0.4, 0.5] if is_high_dim else     # us_crime: máximo 50%
                [0.5, 0.6, 0.7]
            ),
            'bagging_fraction': [0.6, 0.7, 0.8],
            'bagging_freq': [3, 5],
            'reg_alpha': (
                [1.0, 2.0] if (is_small or is_high_dim) else
                [0.1, 0.5, 1.0]
            ),
            'reg_lambda': (
                [2.0, 4.0] if (is_small or is_high_dim) else
                [1.0, 2.0, 4.0]
            )
        },

        # Pipeline: StandardScaler + ElasticNet (Corregimos Underfitting)
        'ElasticNet': {
            'elasticnet__alpha': np.logspace(-4, 1, 25).tolist(), # Valores mucho más bajos para permitir ajuste real
            'elasticnet__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
        }
    }

    return grids.get(nombre_modelo, {})

In [4]:
def calcular_correlacion_errores(errores_dict):
    """
    Calcula la correlación entre errores de pares de modelos.
    Valores bajos indican diversidad.
    """
    nombres = list(errores_dict.keys())
    n = len(nombres)
    correlaciones = np.zeros((n, n))

    for i, m1 in enumerate(nombres):
        for j, m2 in enumerate(nombres):
            if i == j:
                correlaciones[i, j] = 1.0
            else:
                corr = np.corrcoef(errores_dict[m1], errores_dict[m2])[0, 1]
                correlaciones[i, j] = corr

    return pd.DataFrame(correlaciones, index=nombres, columns=nombres)


def procesar_dataset_oraculo(X, y, dataset_name):
    """
    Nested CV con evaluación independiente en test set
    Repeated CV para datasets pequeños
    """
    print(f"\n{'='*60}")
    print(f"🔵 PROCESANDO DATASET: {dataset_name} | Shape: {X.shape}")
    print(f"{'='*60}")

    n_samples, n_features = X.shape
    is_small_dataset = n_samples < 500

    # ===============================
    # SPLIT INICIAL: Train/Test
    # ===============================
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )

    print(f"📊 Split: Train={X_train_full.shape[0]} | Test={X_test.shape[0]}")

    # ===============================
    # CV STRATEGY: Repeated CV para datasets pequeños
    # ===============================
    if is_small_dataset:
        n_splits = 3
        n_repeats = 3
        cv_inner = RepeatedKFold(
            n_splits=n_splits,
            n_repeats=n_repeats,
            random_state=42
        )
        print(f"🔁 CV Strategy: Repeated {n_splits}-Fold x{n_repeats} (dataset pequeño, reduce varianza)")
    else:
        n_splits = 5
        cv_inner = KFold(
            n_splits=n_splits,
            shuffle=True,
            random_state=42
        )
        print(f"🔁 CV Strategy: {n_splits}-Fold CV")

    pool_inicial = get_model_pool()
    nombres_pool = list(pool_inicial.keys())

    n_iter_map = {
        'ExtraTrees': 30,
        'RandomForest': 30,
        'DT-Simple': 18,
        'Ridge': 25,
        'KNN': 25,
        'XGBoost': 25,
        'LightGBM': 25,
        'ElasticNet': 25
    }

    modelos_ajustados_todos = {}
    preds_oof_todos = {}
    diagnostico_general = {}
    cv_std_dict = {}  # Nueva métrica

    print(f"\n1️⃣ Ajustando hiperparámetros (Inner CV)...")

    for name in nombres_pool:

        base_model = get_model_pool()[name]
        param_grid = get_param_grid(name, X_train_full.shape[0], n_features)
        n_iter_modelo = n_iter_map.get(name, 20)

        print(f"\n   -> Ajustando {name} (n_iter={n_iter_modelo})...")

        search = RandomizedSearchCV(
            base_model,
            param_grid,
            n_iter=n_iter_modelo,
            cv=cv_inner,
            scoring='neg_root_mean_squared_error',
            n_jobs=-1,
            random_state=42,
            refit=True,
            return_train_score=True  # Para diagnóstico
        )

        try:
            search.fit(X_train_full, y_train_full)
            best_model = search.best_estimator_

            # ===============================
            # CÁLCULO OOF (sobre train_full)
            # ===============================
            oof_preds = cross_val_predict(
                best_model, X_train_full, y_train_full,
                cv=cv_inner,
                n_jobs=-1
            )

            rmse_oof = np.sqrt(mean_squared_error(y_train_full, oof_preds))

            # ===============================
            # DIAGNÓSTICO OVERFIT (mejorado)
            # ===============================
            cv_scores_train = -search.cv_results_['mean_train_score'][search.best_index_]
            cv_scores_test = -search.cv_results_['mean_test_score'][search.best_index_]
            cv_std_test = search.cv_results_['std_test_score'][search.best_index_]

            gap_ratio = cv_scores_test / cv_scores_train if cv_scores_train > 0 else 1.0

            # Diagnóstico más sofisticado
            if name == 'KNN':
                diagnostico = "⚪ KNN (train score no informativo)"
            elif gap_ratio > 1.25:
                diagnostico = "🔴 OVERFITTING"
            elif gap_ratio > 1.10:
                diagnostico = "🟡 LEVE OVERFITTING"
            elif gap_ratio < 1.05 and cv_scores_test > np.median(list(diagnostico_general.values()) or [cv_scores_test]):
                diagnostico = "🔵 POSIBLE UNDERFITTING"
            else:
                diagnostico = "🟢 AJUSTE SALUDABLE"

            print(f"      RMSE Train (CV): {cv_scores_train:.4f}")
            print(f"      RMSE Valid (CV): {cv_scores_test:.4f} ± {cv_std_test:.4f}")
            print(f"      RMSE OOF      : {rmse_oof:.4f}")
            print(f"      Gap Ratio     : {gap_ratio:.3f}")
            print(f"      Diagnóstico   : {diagnostico}")

            modelos_ajustados_todos[name] = best_model
            preds_oof_todos[name] = oof_preds
            diagnostico_general[name] = cv_scores_test
            cv_std_dict[name] = cv_std_test

        except Exception as e:
            print(f"   ⚠️ Fallo ajustando {name}: {e}")

    if len(modelos_ajustados_todos) < 3:
        print("⚠️ No hay suficientes modelos ajustados para formar trío.")
        return None

    print("\n2️⃣ Analizando diversidad de modelos (correlación de errores)...")

    sq_errors_dict = {}
    rmse_dict = {}

    for name in modelos_ajustados_todos:
        sq_errors_dict[name] = (y_train_full - preds_oof_todos[name])**2
        rmse_dict[name] = np.sqrt(np.mean(sq_errors_dict[name]))

    # Nueva métrica: correlación entre errores
    errores_dict = {name: y_train_full - preds_oof_todos[name] for name in modelos_ajustados_todos}
    df_correlaciones = calcular_correlacion_errores(errores_dict)

    print("\n   📉 Correlación entre errores (valores bajos = más diversidad):")
    print(df_correlaciones.round(3))

    best_single_name = min(rmse_dict, key=rmse_dict.get)
    best_single_rmse = rmse_dict[best_single_name]

    print(f"\n   🏆 Mejor Individual (OOF): {best_single_name} (RMSE: {best_single_rmse:.4f})")

    print("\n3️⃣ Buscando el Dream Team (Oráculo sobre OOF)...")

    # EL ORÁCULO ANCLADO: Obligamos a que el mejor especialista individual esté en el trío
    modelos_restantes = [m for m in modelos_ajustados_todos.keys() if m != best_single_name]
    pares = list(itertools.combinations(modelos_restantes, 2))
    trios = [(best_single_name, p[0], p[1]) for p in pares]
    trio_results = []

    for trio in trios:
        m1, m2, m3 = trio

        trio_sq_errors = np.column_stack((
            sq_errors_dict[m1],
            sq_errors_dict[m2],
            sq_errors_dict[m3]
        ))

        min_sq_errors = np.min(trio_sq_errors, axis=1)
        oracle_rmse = np.sqrt(np.mean(min_sq_errors))
        improvement = 100 * (1 - oracle_rmse / best_single_rmse)

        # Diversidad del trío (correlación promedio entre sus errores)
        corr_trio = [
            df_correlaciones.loc[m1, m2],
            df_correlaciones.loc[m1, m3],
            df_correlaciones.loc[m2, m3]
        ]
        diversidad_promedio = np.mean(corr_trio)

        trio_results.append({
            'Trio': trio,
            'Oracle_RMSE': oracle_rmse,
            'Mejora_%': improvement,
            'Diversidad_Avg_Corr': diversidad_promedio
        })

    df_trio = pd.DataFrame(trio_results).sort_values('Oracle_RMSE')

    dream_team_names = df_trio.iloc[0]['Trio']
    mejor_oracle_rmse = df_trio.iloc[0]['Oracle_RMSE']
    mejora_teorica = df_trio.iloc[0]['Mejora_%']
    diversidad_trío = df_trio.iloc[0]['Diversidad_Avg_Corr']

    print(f"   ✨ Dream Team: {dream_team_names}")
    print(f"   🔮 Mejora Teórica (OOF): {mejora_teorica:.2f}%")
    print(f"   🎲 Diversidad (Corr Promedio): {diversidad_trío:.3f}")

    # ===============================
    # VALIDACIÓN INDEPENDIENTE (TEST)
    # ===============================
    print("\n4️⃣ Evaluación independiente en Test Set...")

    modelos_finales = [modelos_ajustados_todos[name] for name in dream_team_names]

    # Predicciones individuales en test
    preds_test = {}
    rmse_test_individual = {}

    for name in dream_team_names:
        pred_test = modelos_ajustados_todos[name].predict(X_test)
        preds_test[name] = pred_test
        rmse_test_individual[name] = np.sqrt(mean_squared_error(y_test, pred_test))

    # Mejor individual en test
    best_test_name = min(rmse_test_individual, key=rmse_test_individual.get)
    best_test_rmse = rmse_test_individual[best_test_name]

    # Oracle en test (cota superior teórica)
    trio_sq_errors_test = np.column_stack([
        (y_test - preds_test[name])**2 for name in dream_team_names
    ])
    min_sq_errors_test = np.min(trio_sq_errors_test, axis=1)
    oracle_rmse_test = np.sqrt(np.mean(min_sq_errors_test))
    mejora_real_test = 100 * (1 - oracle_rmse_test / best_test_rmse)

    print(f"   🧪 Mejor Individual (Test): {best_test_name} -> RMSE = {best_test_rmse:.4f}")
    print(f"   🧪 Oracle Score (Test)   : RMSE = {oracle_rmse_test:.4f}")
    print(f"   📈 Mejora Real (Test)    : {mejora_real_test:.2f}%")

    # ===============================
    # GUARDADO
    # ===============================
    ruta_archivo = f"{DIRECTORIO_SALIDA}/{dataset_name}_best_models.pkl"

    datos_a_guardar = {
        'modelos': modelos_finales,
        'nombres': dream_team_names,
        'oracle_rmse_oof': mejor_oracle_rmse,
        'oracle_rmse_test': oracle_rmse_test,
        'mejora_teorica_oof': mejora_teorica,
        'mejora_real_test': mejora_real_test,
        'diversidad': diversidad_trío,
        'correlaciones': df_correlaciones,
        'cv_std': cv_std_dict
    }

    joblib.dump(datos_a_guardar, ruta_archivo)

    print(f"\n   ✅ Dream Team guardado en: {ruta_archivo}")

    return {
        'Dataset': dataset_name,
        'N_samples': n_samples,
        'N_features': n_features,
        'CV_Strategy': f"Repeated {n_splits}x{n_repeats}" if is_small_dataset else f"{n_splits}-Fold",
        'Mejor_Individual_OOF': best_single_name,
        'RMSE_Individual_OOF': best_single_rmse,
        'Dream_Team': " + ".join(dream_team_names),
        'Oracle_RMSE_OOF': mejor_oracle_rmse,
        'Mejora_Teorica_OOF_%': mejora_teorica,
        'Oracle_RMSE_Test': oracle_rmse_test,
        'Mejora_Real_Test_%': mejora_real_test,
        'Diversidad_Avg_Corr': diversidad_trío,
        'CV_Std_Promedio': np.mean(list(cv_std_dict.values()))
    }

In [5]:
archivos_datasets = glob.glob("../data/regression/*.arff")
resultados_globales = []

if not archivos_datasets:
    print("¡Ojo! No se han encontrado archivos .arff.")
else:
    for ruta in tqdm(archivos_datasets, desc="Procesando datasets"):
        nombre_ds = os.path.basename(ruta).replace('.arff', '')
        X, y = LoadData(ruta)

        if X is not None:
            # Asegurar compatibilidad estricta de memoria para Numpy/C++
            if isinstance(X, pd.DataFrame):
                X = np.ascontiguousarray(X.values, dtype=np.float64)
            y = np.ascontiguousarray(y, dtype=np.float64)

            # Ejecutar y acumular el dict resultante
            metricas_dataset = procesar_dataset_oraculo(X, y, nombre_ds)

            if metricas_dataset is not None:
                resultados_globales.append(metricas_dataset)

    # 5. Guardar CSV resumen (Fuera del bucle for, pero dentro del else)
    if resultados_globales:
        df_resumen = pd.DataFrame(resultados_globales)
        ruta_csv = os.path.join(DIRECTORIO_SALIDA, "resumen_dream_teams.csv")
        df_resumen.to_csv(ruta_csv, index=False)
        print(f"\n📊 Ejecución finalizada. Resumen global guardado en: {ruta_csv}")
        print("\n" + "="*80)
        print("📋 RESUMEN FINAL:")
        print("="*80)
        print(df_resumen.to_string(index=False))


Procesando datasets:   0%|          | 0/5 [00:00<?, ?it/s]


🔵 PROCESANDO DATASET: abalone | Shape: (4177, 7)
📊 Split: Train=3341 | Test=836
🔁 CV Strategy: 5-Fold CV

1️⃣ Ajustando hiperparámetros (Inner CV)...

   -> Ajustando ExtraTrees (n_iter=30)...
      RMSE Train (CV): 2.2812
      RMSE Valid (CV): 2.3276 ± 0.0742
      RMSE OOF      : 2.3288
      Gap Ratio     : 1.020
      Diagnóstico   : 🟢 AJUSTE SALUDABLE

   -> Ajustando RandomForest (n_iter=30)...
      RMSE Train (CV): 2.0162
      RMSE Valid (CV): 2.1770 ± 0.0753
      RMSE OOF      : 2.1783
      Gap Ratio     : 1.080
      Diagnóstico   : 🟢 AJUSTE SALUDABLE

   -> Ajustando DT-Simple (n_iter=18)...
      RMSE Train (CV): 2.1088
      RMSE Valid (CV): 2.2907 ± 0.0794
      RMSE OOF      : 2.2921
      Gap Ratio     : 1.086
      Diagnóstico   : 🟢 AJUSTE SALUDABLE

   -> Ajustando Ridge (n_iter=25)...
      RMSE Train (CV): 2.2042
      RMSE Valid (CV): 2.2523 ± 0.0870
      RMSE OOF      : 2.2540
      Gap Ratio     : 1.022
      Diagnóstico   : 🟢 AJUSTE SALUDABLE

   -> Ajusta

Procesando datasets:  20%|██        | 1/5 [00:06<00:24,  6.13s/it]

      RMSE Train (CV): -0.0000
      RMSE Valid (CV): 2.1960 ± 0.0859
      RMSE OOF      : 2.1977
      Gap Ratio     : 1.000
      Diagnóstico   : ⚪ KNN (train score no informativo)

2️⃣ Analizando diversidad de modelos (correlación de errores)...

   📉 Correlación entre errores (valores bajos = más diversidad):
              ExtraTrees  RandomForest  DT-Simple  Ridge    KNN
ExtraTrees         1.000         0.958      0.893  0.883  0.954
RandomForest       0.958         1.000      0.948  0.900  0.969
DT-Simple          0.893         0.948      1.000  0.855  0.906
Ridge              0.883         0.900      0.855  1.000  0.891
KNN                0.954         0.969      0.906  0.891  1.000

   🏆 Mejor Individual (OOF): RandomForest (RMSE: 2.1783)

3️⃣ Buscando el Dream Team (Oráculo sobre OOF)...
   ✨ Dream Team: ('RandomForest', 'DT-Simple', 'Ridge')
   🔮 Mejora Teórica (OOF): 16.75%
   🎲 Diversidad (Corr Promedio): 0.901

4️⃣ Evaluación independiente en Test Set...
   🧪 Mejor Indivi

Procesando datasets:  40%|████      | 2/5 [00:08<00:11,  3.97s/it]

...

      RMSE Train (CV): 0.6101
      RMSE Valid (CV): 8.3760 ± 0.6768
      RMSE OOF      : 8.4039
      Gap Ratio     : 13.729
      Diagnóstico   : ⚪ KNN (train score no informativo)

2️⃣ Analizando diversidad de modelos (correlación de errores)...

   📉 Correlación entre errores (valores bajos = más diversidad):
              ExtraTrees  RandomForest  DT-Simple  Ridge    KNN
ExtraTrees         1.000         0.860      0.629  0.753  0.655
RandomForest       0.860         1.000      0.823  0.609  0.551
DT-Simple          0.629         0.823      1.000  0.433  0.381
Ridge              0.753         0.609      0.433  1.000  0.620
KNN                0.655         0.551      0.381  0.620  1.000

   🏆 Mejor Individual (OOF): KNN (RMSE: 8.4039)

3️⃣ Buscando el Dream Team (Oráculo sobre OOF)...
   ✨ Dream Team: ('KNN', 'DT-Simple', 'Ridge')
   🔮 Mejora Teórica (OOF): 44.54%
   🎲 Diversidad (Corr Promedio): 0.478

4️⃣ Evaluación independiente en Test Set...
   🧪 Mejor Individual (Test): KNN -

Procesando datasets:  80%|████████  | 4/5 [00:36<00:12, 12.09s/it]

   🧪 Mejor Individual (Test): Ridge -> RMSE = 0.0028
   🧪 Oracle Score (Test)   : RMSE = 0.0021
   📈 Mejora Real (Test)    : 25.20%

   ✅ Dream Team guardado en: ../modelos_ajustados/elevators_best_models.pkl

🔵 PROCESANDO DATASET: us_crime | Shape: (1994, 126)
📊 Split: Train=1595 | Test=399
🔁 CV Strategy: 5-Fold CV

1️⃣ Ajustando hiperparámetros (Inner CV)...

   -> Ajustando ExtraTrees (n_iter=30)...
      RMSE Train (CV): 0.1318
      RMSE Valid (CV): 0.1415 ± 0.0090
      RMSE OOF      : 0.1418
      Gap Ratio     : 1.074
      Diagnóstico   : 🟢 AJUSTE SALUDABLE

   -> Ajustando RandomForest (n_iter=30)...
      RMSE Train (CV): 0.1257
      RMSE Valid (CV): 0.1398 ± 0.0092
      RMSE OOF      : 0.1401
      Gap Ratio     : 1.112
      Diagnóstico   : 🟡 LEVE OVERFITTING

   -> Ajustando DT-Simple (n_iter=18)...
      RMSE Train (CV): 0.1330
      RMSE Valid (CV): 0.1525 ± 0.0093
      RMSE OOF      : 0.1528
      Gap Ratio     : 1.147
      Diagnóstico   : 🟡 LEVE OVERFITTING

   ->

Procesando datasets: 100%|██████████| 5/5 [00:44<00:00,  8.94s/it]

      RMSE Train (CV): -0.0000
      RMSE Valid (CV): 0.1469 ± 0.0070
      RMSE OOF      : 0.1471
      Gap Ratio     : 1.000
      Diagnóstico   : ⚪ KNN (train score no informativo)

2️⃣ Analizando diversidad de modelos (correlación de errores)...

   📉 Correlación entre errores (valores bajos = más diversidad):
              ExtraTrees  RandomForest  DT-Simple  Ridge    KNN
ExtraTrees         1.000         0.988      0.884  0.906  0.898
RandomForest       0.988         1.000      0.886  0.904  0.898
DT-Simple          0.884         0.886      1.000  0.820  0.802
Ridge              0.906         0.904      0.820  1.000  0.851
KNN                0.898         0.898      0.802  0.851  1.000

   🏆 Mejor Individual (OOF): Ridge (RMSE: 0.1382)

3️⃣ Buscando el Dream Team (Oráculo sobre OOF)...
   ✨ Dream Team: ('Ridge', 'DT-Simple', 'KNN')
   🔮 Mejora Teórica (OOF): 24.33%
   🎲 Diversidad (Corr Promedio): 0.824

4️⃣ Evaluación independiente en Test Set...
   🧪 Mejor Individual (Test): Rid